In [1]:
%pip install neo4j


   ---------------------------------------- 0.0/312.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/312.3 kB ? eta -:--:--
   --- ----------------------------------- 30.7/312.3 kB 660.6 kB/s eta 0:00:01
   -------- ------------------------------ 71.7/312.3 kB 653.6 kB/s eta 0:00:01
   ------------ ------------------------- 102.4/312.3 kB 653.6 kB/s eta 0:00:01
   ------------ ------------------------- 102.4/312.3 kB 653.6 kB/s eta 0:00:01
   ------------ ------------------------- 102.4/312.3 kB 653.6 kB/s eta 0:00:01
   ------------- ------------------------ 112.6/312.3 kB 364.4 kB/s eta 0:00:01
   -------------- ----------------------- 122.9/312.3 kB 379.3 kB/s eta 0:00:01
   -------------- ----------------------- 122.9/312.3 kB 379.3 kB/s eta 0:00:01
   ----------------------------- -------- 245.8/312.3 kB 558.1 kB/s eta 0:00:01
   ----------------------------- -------- 245.8/312.3 kB 558.1 kB/s eta 0:00:01
   -------------------------------- ----- 266.2/312.3 kB 5


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# #cases, acts reference , but no all the relevant acts displaying for a case
# # 📦 Required imports
# import json
# import re
# import pandas as pd
# from neo4j import GraphDatabase

# # 🔹 Step 1: Load JSON file
# file_path = r"C:\Users\User\Desktop\Legal-Research-Platform-Core\resources\cases_2024.json"
# with open(file_path, 'r', encoding='utf-8') as file:
#     case_data = json.load(file)

# # 🔹 Step 2: Regular expression patterns for act references
# act_patterns = [
#     r"Bail\s+Act\s+No\.\s*\d+\s*of\s*\d+",
#     r"Penal\s+Code\s*\(Amendment\)\s*Act\s+No\.\s*\d+\s*of\s*\d+",
#     r"Evidence\s*\(Special\s*Provisions\)\s*Act\s+No\.\s*\d+",
#     r"Code\s+of\s+Criminal\s+Procedure\s*Act\s+No\.\s*\d+",
#     r"Constitution\s+of\s+the\s+Democratic\s+Socialist\s+Republic\s+of\s+Sri\s+Lanka",
#     r"Criminal\s+Procedure\s+Code\s+Act\s+No\.\s*\d+",
#     r"Children\s+and\s+Young\s+Persons\s+Ordinance\s*Cap\.\s*\d+"
# ]
# compiled_patterns = [re.compile(p, re.IGNORECASE) for p in act_patterns]

# # 🔹 Step 3: Extract acts per case
# case_act_map = []
# for case in case_data:
#     case_id = case.get("id", "Unknown")
#     content = case.get("text", "")

#     acts_found = set()
#     for pattern in compiled_patterns:
#         matches = pattern.findall(content)
#         for match in matches:
#             acts_found.add(match.strip())

#     case_act_map.append({
#         "case_id": case_id,
#         "acts": list(acts_found)
#     })

# # 🔹 Step 4: Save & Preview
# df = pd.DataFrame(case_act_map)
# pd.set_option('display.max_colwidth', None)
# from IPython.display import display
# display(df)
# df.to_csv("case_to_acts_array.csv", index=False, encoding="utf-8")

# # 🔹 Step 5: Neo4j connector class
# class Neo4jConnector:
#     def __init__(self, uri, user, password):
#         self.driver = GraphDatabase.driver(uri, auth=(user, password))

#     def close(self):
#         self.driver.close()

#     def push_case_with_acts(self, case_id, acts):
#         with self.driver.session() as session:
#             session.execute_write(self._create_graph_tx, case_id, acts)

#     @staticmethod
#     def _create_graph_tx(tx, case_id, acts):
#         # Create only case_id property — no filename
#         tx.run(
#             """
#             MERGE (c:Case {id: $case_id})
#             SET c.case_id = $case_id
#             """, case_id=case_id
#         )

#         for act in acts:
#             tx.run(
#                 """
#                 MERGE (a:Act {name: $act})
#                 WITH a
#                 MATCH (c:Case {id: $case_id})
#                 MERGE (c)-[:REFERS_TO]->(a)
#                 """, case_id=case_id, act=act
#             )

# # 🔹 Step 6: Connect to Neo4j
# neo4j_conn = Neo4jConnector(uri="bolt://localhost:7687", user="neo4j", password="neo4gS@123")

# # 🔄 Step 7: Insert all cases + acts
# for entry in case_act_map:
#     if entry["acts"]:
#         neo4j_conn.push_case_with_acts(entry["case_id"], entry["acts"])

# # ✅ Step 8: Close connection
# neo4j_conn.close()


In [1]:
#cases, acts reference 
# 📦 Required imports
import json
import re
import pandas as pd
from neo4j import GraphDatabase
from IPython.display import display

# 📁 🔹 Step 1: Load JSON file
# CHANGE THIS LINE to point to your file path
file_path = r"C:\Users\User\Desktop\Legal-Research-Platform-Core\resources\cases_2024.json"

with open(file_path, 'r', encoding='utf-8') as file:
    case_data = json.load(file)

# 🔍 🔹 Step 2: Enhanced regex pattern for Act references
# This pattern captures many variants like:
# Penal Code Act No. 15 of 1979, Bail Act No. 30 of 1997, etc.
act_pattern = re.compile(
    r'\b(?:[A-Z][a-z]+\s+){0,5}(Act|Code|Ordinance|Law|Regulation)(?:\s*\(.*?\))?\s+No\.?\s*\d+\s*(?:of\s*\d{4})?',
    re.IGNORECASE
)

# 📊 🔹 Step 3: Extract acts per case
case_act_map = []

for case in case_data:
    case_id = case.get("id", "Unknown")
    content = case.get("text", "")

    # Normalize and search
    matches = act_pattern.findall(content)
    all_matches = act_pattern.finditer(content)

    acts_found = set()
    for match in all_matches:
        act_text = match.group(0).strip()
        acts_found.add(act_text)

    case_act_map.append({
        "case_id": case_id,
        "acts": sorted(acts_found)  # sort for readability
    })

# 📁 🔹 Step 4: Save & Preview
df = pd.DataFrame(case_act_map)
pd.set_option('display.max_colwidth', None)
display(df)

# Save to CSV
df.to_csv("case_to_acts_extracted.csv", index=False, encoding="utf-8")

# 🧠 🔹 Step 5: Neo4j connector class
class Neo4jConnector:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def push_case_with_acts(self, case_id, acts):
        with self.driver.session() as session:
            session.execute_write(self._create_graph_tx, case_id, acts)

    @staticmethod
    def _create_graph_tx(tx, case_id, acts):
        # Create Case node
        tx.run(
            """
            MERGE (c:Case {id: $case_id})
            SET c.case_id = $case_id
            """, case_id=case_id
        )

        # Create Act nodes + relationships
        for act in acts:
            tx.run(
                """
                MERGE (a:Act {name: $act})
                WITH a
                MATCH (c:Case {id: $case_id})
                MERGE (c)-[:REFERS_TO]->(a)
                """, case_id=case_id, act=act
            )

# ✅ 🔹 Step 6: Connect and push to Neo4j
neo4j_conn = Neo4jConnector(
    uri="bolt://localhost:7687",
    user="neo4j",
    password="neo4gS@123"  # change if needed
)

# Insert all case-acts into the graph
for entry in case_act_map:
    if entry["acts"]:
        neo4j_conn.push_case_with_acts(entry["case_id"], entry["acts"])

# 🔚 🔹 Step 7: Close connection
neo4j_conn.close()


,case_id,acts
0,d66a6895-c339-4bd0-9992-790b7b5f4a17,"[ACT NO. 2 of 1995, RELEVANT \nprovistions of the bail ACT NO. 30 of 1997]"
1,4aaafdf5-8ac9-4086-b62e-485d250b02bb,"[Act \nNo.32 of 1999, Act No 32 of 1999, Act No. 32 of 1999, Act No.32 of 1999, the Code of Criminal \nProcedure Act No.15 of 1979, the Penal Code as \namended Act No.22 of 0995]"
2,f81236b6-7c88-4337-9701-772651a56abe,[]
3,0fe6fe07-fd7d-4b4c-a5d3-3644e9c56b56,[]
4,b655451f-cad0-4cc4-b5ce-6bc81dbbee30,[]
...,...,...
524,ac7fca87-465f-4a22-b064-a4fa1ffcefc1,[the Code of Criminal \nProcedure Act No.15 of 1979]
525,4c4ebba0-1876-43ba-adf6-79afca98fceb,"[Protection Ordinance as amended by Act No. 22 of 2009, Protection Ordinance as \namended by Act No.22 of 2009]"
526,a0079550-0b33-46e2-a360-f01edecb103c,[the Code of Criminal \nProcedure Act No.15 of 1979]
527,313e5fae-828a-40d0-ac21-489a6e6d05d4,"[in terms \nof the Companies Act No. 117 of 1982, ratuity Act No. 12 of \n1983, ratuity Act No. 12 of 1983, registered in terms of Companies Act No. 7 of 2007, requirement under \npayment of Gratuity Act No.12 of 1983]"
